## Electronic Band Structure
---

Crystalline solids have multiple peculiarities connected with their fundamental property: discrete translational symmetry. Additionally, they exhibit additional symmetries which influence their properties (e.g. electronic band structure) as well as procedures used in the calculation. Dealing with crystals require use of Periodic Boundary Conditions and, often, reciprocal space, Brillouin Zone etc. ASE provides multiple tools for working with crystals. We will use some of them in the following examples - which is modeled after one of the AbInit tutorials.

In the following exercise we will:
* build a crystal of Silicon,
* calculate its lattice parameters,
* derive its electronic band structure.

As usual the detailed technical documentation regarding ASE toolbox can be found [ASE documentation site](https://wiki.fysik.dtu.dk/ase/index.html). The documentation concerning the DFT tool used in the exercise is located at [AbInit documentation site](https://docs.abinit.org/).

In [ ]:
%pylab inline
from ase.build import bulk
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.calculators.abinit import Abinit
import ase.io
from ase import units as un
import os

### Building the Silicon crystal

The silicon has a simple crystal structure with two atoms per primitive unit cell - in fact it is the same structure as diamond. Thus, building the structure is almost trivial - the only free parameter is the length of the lattice vector $a$.

In [ ]:
Si = bulk('Si', crystalstructure='diamond', a=5.446 )#a=5.46931040)
print(Si.cell.cellpar())

### Checking the structure

In [ ]:
calc = Abinit(label='Si', directory="work_05", v8_legacy_format=True,
              ecut= 350,
              pps='psp8',
              toldfe=1.0e-6,
              diemac=12,
              nstep=20,
              kptopt=1,
              ngkpt=[4,4,4],
              nshiftk=4,
              shiftk=[[0.5, 0.5, 0.5],
                      [0.5, 0.0, 0.0],
                      [0.0, 0.5, 0.0],
                      [0.0, 0.0, 0.5]]
              )

Si.set_calculator(calc)
print(Si.get_potential_energy(), Si.get_stress()/un.GPa)

### Running Self-Consistent DFT

In [ ]:
Si.calc.set(prtden=1)
Si.get_potential_energy()

### Calculating band structure from SCF electronic density

In [ ]:
# Remove SCF parameter from the calculator
try:
    del Si.calc.parameters['toldfe']
except KeyError:
    pass

In [ ]:
# Define reciprocal space path
kpnts = [[1/2,0,0],[0,0,0],[0,1/2,1/2],[1,1,1]]
kplbl = ['L', '$\Gamma$', 'X', '$\Gamma$']

In [ ]:
# Modify the calculator
Si.calc.set(iscf=-2, getden=-1, nband=8, kptopt=-3, tolwfr=1e-10,
            ndivsm=20, nstep=20, kptbounds=kpnts)
Si.calc.atoms=Si

In [ ]:
# Copy SCF output charge density to input file
!cp work_05/Sio_DEN work_05/Sii_DEN

In [ ]:
# Run non-SCF calculation
Si.calc.calculate()

### Plotting the band structure

In [ ]:
# Extract the reciprocal space path
pth = Si.calc.results['ibz_kpoints']
pth = array([0]+list(norm(pth[1:]-pth[:-1], axis=1))).cumsum()

# Define valence and conduction band indexes
val = 3
cond = 4

# Extract bands and find Fermi level
ev = Si.calc.results['eigenvalues'][0].T
Efermi = ev[val].max()

In [ ]:
# Plot bands
figsize(10,6)
for bnd in ev:
    plot(pth, bnd-Efermi)

# Mark fermi energy    
axhline(ls=":", lw=1)

# Mark high symmetry points
hsp = array(kpnts)
hsp = cumsum([0]+[norm(v) for v in hsp[1:]-hsp[:-1]])
for d in hsp[1:-1]:
    axvline(d, ls=":")  
    
# Decorate the plot
xticks(hsp, kplbl);
title(f'Si: Indirect band gap: {ev[cond].min()-ev[val].max():.2f} eV')
xlim(pth[0], pth[-1])

# Find and mark the gap transition
frm = ev[val].argmax()
to = ev[cond].argmin()
annotate("", xy=(pth[to], ev[4][to]-Efermi), 
         xytext=(pth[frm], ev[3][frm]-Efermi), 
         arrowprops=dict(arrowstyle="->"))
ylabel('Energy (eV)')
xlabel('Wave vector')
savefig('Si_bs_abinit.pdf')